# Python and BigQuery SQL
@johanesalxd

## Preparation

*   For this demo, you can download the audio files from [here](https://www.voiptroubleshooter.com/open_speech/american.html).
  * Original audio transcription can be found [here](https://www.cs.columbia.edu/~hgs/audio/harvard.html).
*   Upload the audio files to your own GCS bucket (click [here](https://cloud.google.com/storage/docs/uploading-objects) for more details).
  * For more details on GCS bucket creation, click [here](https://cloud.google.com/storage/docs/creating-buckets).
  * Get the path of the bucket/folder and update the parameter below.
* Make sure you've all the permission ready, please check the details below:
  * Cloud Function, BigQuery Resource Connection and Remote Function (click [here](https://cloud.google.com/bigquery/docs/remote-functions-translation-tutorial#required_roles_for_your_account)).
  * BigQuery Remote Model (click [here](https://cloud.google.com/bigquery/docs/generate-text#required_permissions)).
* Please refer [here](https://github.com/googleapis/python-bigquery-dataframes) for more information on BigFrames.



## Parameters and Environment Setup

In [1]:
# [ACTION] Please update all parameters below accordingly

# Regions details
LOCATION = "us"
REGION = "us-central1"

# GCS, Cloud Function and Remote Model details
GCS_URI = "gs://johanesa-temporary-us/audio-files/*"
CFN_NAME = "transcription-handler"
MDL_NAME = "gemini-pro"

# BigQuery details
BQ_DATASET = "demo_dataset"
BQ_FN_NAME = "transcribe_audio"
BQ_CONN_NAME = "gcs-connection"
BQ_OBJ_TBL_NAME = "sample_speech"
BQ_SUM_TBL_NAME = "sample_speech_to_text"

In [2]:
# Install additional packages
!sudo apt-get install jq -y

!pip install bigframes

%load_ext google.cloud.bigquery

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 357 kB of archives.
After this operation, 1,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libonig5 amd64 6.9.7.1-2build1 [172 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjq1 amd64 1.6-2.1ubuntu3 [133 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 jq amd64 1.6-2.1ubuntu3 [52.5 kB]
Fetched 357 kB in 1s (358 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize 

## Cloud Function Setup

In [3]:
# Directory setup
FOLDER = "./cf_transcription-handler"
PYTHON_SCRIPT = f"{FOLDER}/main.py"
PYTHON_REQUIREMENTS = f"{FOLDER}/requirements.txt"

!rm -rf {FOLDER} || True
!mkdir {FOLDER}

### Cloud Function Code

In [4]:
# Write dependencies to file (requirements.txt)
%%writefile {PYTHON_REQUIREMENTS}

Flask==2.2.2
functions-framework==3.3.0
google-cloud-speech==2.21.0
Werkzeug==2.3.7

Writing ./cf_transcription-handler/requirements.txt


In [5]:
# Write code to file (main.py)
## [ACTION] Please adjust the config parameter (encoding and sample_rate_hertz) accordingly
%%writefile {PYTHON_SCRIPT}

import flask
import functions_framework
import json

from google.cloud import speech_v1

config = speech_v1.RecognitionConfig(
        encoding=speech_v1.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=8000,
        language_code="en-US",
    )

# Register an HTTP function with the Functions Framework
@functions_framework.http
def audio_transcribe(request: flask.Request) -> flask.Response:
    """BigQuery remote function to label input images.
    Args:
        request: HTTP request from BigQuery
        https://cloud.google.com/bigquery/docs/reference/standard-sql/remote-functions#input_format
    Returns:
        HTTP response to BigQuery
        https://cloud.google.com/bigquery/docs/reference/standard-sql/remote-functions#output_format
    """
    try:
        client = speech_v1.SpeechClient()
        calls = request.get_json()["calls"]
        transcript = []

        for call in calls:
            audio = speech_v1.RecognitionAudio(uri=call[0])
            operation = client.long_running_recognize(config=config, audio=audio)

            print("Waiting for operation to complete...")
            response = operation.result(timeout=90)

            transcript_builder = {"transcript":[],"confidence":[]}
            # Each result is for a consecutive portion of the audio. Iterate through
            # them to get the transcripts for the entire audio file.
            for result in response.results:
                # The first alternative is the most likely one for this portion.
                transcript_builder["transcript"].append(result.alternatives[0].transcript)
                transcript_builder["confidence"].append(result.alternatives[0].confidence)
            transcript_builder["transcript"] = "".join(transcript_builder["transcript"])
            transcript.append(json.dumps(transcript_builder))

        return flask.make_response(flask.jsonify({"replies": transcript}))
    except Exception as e:
        return flask.make_response(flask.jsonify({"errorMessage": str(e)}), 400)

Writing ./cf_transcription-handler/main.py


### Cloud Function Deployment

In [ ]:
# Deploy code to Cloud Function
## [ACTION] Please adjust the entry point accordingly
## Unauthenticated connection is allowed for this demo
!gcloud functions deploy {CFN_NAME} \
--gen2 \
--runtime=python310 \
--region={REGION} \
--source={FOLDER} \
--entry-point=audio_transcribe \
--trigger-http \
--allow-unauthenticated

## BigQuery Setup


In [9]:
# Import BigQuery Client (as an alternative of BigQuery magics due to parameterized queries compatibility)
from google.cloud import bigquery

client = bigquery.Client()

### BigQuery Resource Connection Setup


In [ ]:
# Create Cloud Resource Connection
## Will trigger an error if connection already exist
!bq mk --connection --location={LOCATION} --connection_type=CLOUD_RESOURCE {BQ_CONN_NAME}

In [ ]:
# Get the Service Account and grant Cloud Functions Invoker roles
BQ_SA = !bq show --connection --format json {LOCATION}.{BQ_CONN_NAME} | jq -r '.cloudResource.serviceAccountId'

!gcloud functions add-invoker-policy-binding transcription-handler \
      --region="{REGION}" \
      --member="serviceAccount:{BQ_SA[0]}"

### BigQuery Remote Function Setup

In [10]:
# Create command for BigQuery Functions
## Get Cloud Function endpoint
CFN_ENDPOINT = !gcloud functions describe {CFN_NAME} --gen2 --region {REGION} --format="value(url)"

query = ('''
CREATE OR REPLACE FUNCTION
  `{BQ_DATASET}.{BQ_FN_NAME}`(x STRING)
  RETURNS STRING REMOTE
WITH CONNECTION `{LOCATION}.{BQ_CONN_NAME}` OPTIONS (endpoint = '{CFN_ENDPOINT[0]}',
    max_batching_rows = 5)
;
''').format(BQ_DATASET=BQ_DATASET, BQ_FN_NAME=BQ_FN_NAME, BQ_CONN_NAME=BQ_CONN_NAME, \
            CFN_ENDPOINT=CFN_ENDPOINT, LOCATION=LOCATION)

client.query(query).result()

In [11]:
# Create command for BigQuery Remote Model (Gemini Pro)
query = (
    """
CREATE OR REPLACE MODEL
  `{BQ_DATASET}.{MDL_NAME}` REMOTE
WITH CONNECTION `{LOCATION}.{BQ_CONN_NAME}` OPTIONS (ENDPOINT = '{MDL_NAME}')
;
"""
).format(
    BQ_DATASET=BQ_DATASET,
    BQ_CONN_NAME=BQ_CONN_NAME,
    LOCATION=LOCATION,
    MDL_NAME=MDL_NAME,
)

client.query(query).result()

In [13]:
# Create command to build baseline table (BigQuery Object Tables)
## Will trigger an error if table already exist
query = (
    """
CREATE OR REPLACE EXTERNAL TABLE
  {BQ_DATASET}.{BQ_OBJ_TBL_NAME}
WITH CONNECTION `{LOCATION}.{BQ_CONN_NAME}` OPTIONS( object_metadata = 'SIMPLE',
    uris = ['{GCS_URI}'])
;
"""
).format(
    BQ_DATASET=BQ_DATASET,
    BQ_OBJ_TBL_NAME=BQ_OBJ_TBL_NAME,
    BQ_CONN_NAME=BQ_CONN_NAME,
    GCS_URI=GCS_URI,
    CFN_ENDPOINT=CFN_ENDPOINT,
    LOCATION=LOCATION,
)

client.query(query).result()

## BigQuery Analysis Example

### BigQuery Speech to Text

In [14]:
# Create command to query baseline table
## Uncomment to execute
query = (
    """
SELECT
  *
FROM
  `{BQ_DATASET}.{BQ_OBJ_TBL_NAME}`
LIMIT
  100
;
"""
).format(BQ_DATASET=BQ_DATASET, BQ_OBJ_TBL_NAME=BQ_OBJ_TBL_NAME)

client.query(query).to_dataframe()

,uri,generation,content_type,size,md5_hash,updated,metadata
0,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429523842832,audio/wav,538014,558b1f681c59f204e402d560956a63a5,2023-09-11 10:52:03.889000+00:00,[]
1,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429523832483,audio/wav,524600,105075551b4f535fc54013b006dba01d,2023-09-11 10:52:03.872000+00:00,[]
2,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429523079561,audio/wav,532338,2ee2c4b96d3d5c94e13b8c33a0577241,2023-09-11 10:52:03.133000+00:00,[]
3,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429523072273,audio/wav,520730,1879ee3d3a133591fc158bb5fa19f92b,2023-09-11 10:52:03.122000+00:00,[]
4,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429523066648,audio/wav,498000,59ac35ead40460c6613db87811262fb0,2023-09-11 10:52:03.106000+00:00,[]
5,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429728115429,audio/wav,662836,57c12a456073baa9b28f71d21ffa3d53,2023-09-11 10:55:28.167000+00:00,[]
6,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429728115130,audio/wav,562104,657d3fe7556ffa9bc2455ff6eb81b332,2023-09-11 10:55:28.151000+00:00,[]
7,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429727352967,audio/wav,568126,e4b41ada79f3ca6497eefc4389ec64f7,2023-09-11 10:55:27.407000+00:00,[]
8,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429727341600,audio/wav,553294,b837b6cf6076d0bf3905fe219862109b,2023-09-11 10:55:27.391000+00:00,[]
9,gs://johanesa-temporary-us/audio-files/OSR_us_...,1694429727335125,audio/wav,517040,09ec71f52675b0dd3c143e95defbdfb9,2023-09-11 10:55:27.376000+00:00,[]


In [15]:
# Create command to build summary table
query = (
    """
CREATE OR REPLACE TABLE
  `{BQ_DATASET}.{BQ_SUM_TBL_NAME}` AS
WITH
  a AS (
  SELECT
    uri,
    {BQ_DATASET}.{BQ_FN_NAME}(uri) AS transcribe_result,
  FROM
    `{BQ_DATASET}.{BQ_OBJ_TBL_NAME}`)
SELECT
  uri,
  JSON_EXTRACT(transcribe_result, "$.transcript") AS transcribe_extract
FROM
  a
;
"""
).format(
    BQ_DATASET=BQ_DATASET,
    BQ_OBJ_TBL_NAME=BQ_OBJ_TBL_NAME,
    BQ_SUM_TBL_NAME=BQ_SUM_TBL_NAME,
    BQ_FN_NAME=BQ_FN_NAME,
)

client.query(query).result()

In [16]:
# Create command to query summary table
query = (
    """
SELECT
  *
FROM
  `{BQ_DATASET}.{BQ_SUM_TBL_NAME}`
LIMIT
  100
;
"""
).format(BQ_DATASET=BQ_DATASET, BQ_SUM_TBL_NAME=BQ_SUM_TBL_NAME)

client.query(query).to_dataframe()

,uri,transcribe_extract
0,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the small Pub not a hole in the sock this is ..."
1,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the boy was there when the sun rose hey Rod i..."
2,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the Birch canoes slid on the smooth planks gl..."
3,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""who is the low to left shoulder take the wind..."
4,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the King rule the state in the early days thi..."
5,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the small pop not a hole in the sock the fish..."
6,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""King rules estate in early days the ship was ..."
7,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""who is the low to your left shoulder take the..."
8,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the boy was there when the sun rose hey Rod i..."
9,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the bush can use slid on the smooth planks gl..."


### Using BigQuery Magic

In [17]:
%%bigquery
SELECT
  *
FROM
  `demo_dataset.sample_speech_to_text`
LIMIT
  100
;


Query is running:   0%|          |

Downloading:   0%|          |

,uri,transcribe_extract
0,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the small Pub not a hole in the sock this is ..."
1,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the boy was there when the sun rose hey Rod i..."
2,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the Birch canoes slid on the smooth planks gl..."
3,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""who is the low to left shoulder take the wind..."
4,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the King rule the state in the early days thi..."
5,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the small pop not a hole in the sock the fish..."
6,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""King rules estate in early days the ship was ..."
7,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""who is the low to your left shoulder take the..."
8,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the boy was there when the sun rose hey Rod i..."
9,gs://johanesa-temporary-us/audio-files/OSR_us_...,"""the bush can use slid on the smooth planks gl..."


### BigQuery Sentiment Analysis

In [ ]:
# Create command to do sentiment analysis using Remote Model
query = (
    """
SELECT
  prompt,
  JSON_EXTRACT(ml_generate_text_result, "$.candidates[0].content.parts[0].text") AS sentiment
FROM
  ML.GENERATE_TEXT( MODEL `{BQ_DATASET}.{MDL_NAME}`,
    (
    SELECT
      CONCAT( 'Classify the sentiment of the following text as positive or negative.Text:', transcribe_extract, 'Sentiment:') AS prompt
    FROM
      `{BQ_DATASET}.{BQ_SUM_TBL_NAME}`
    LIMIT
      3 ),
    STRUCT( 0.1 AS temperature,
      1000 AS max_output_tokens,
      0.1 AS top_p,
      10 AS top_k))
;
"""
).format(BQ_DATASET=BQ_DATASET, BQ_SUM_TBL_NAME=BQ_SUM_TBL_NAME, MDL_NAME=MDL_NAME)

client.query(query).to_dataframe()

,prompt,sentiment
0,Classify the sentiment of the following text a...,"""Negative"""
1,Classify the sentiment of the following text a...,"""Positive"""
2,Classify the sentiment of the following text a...,"""Neutral"""
